In [1]:
#!pip install -U ray

In [2]:
#!pip install -U "ray[tune]"  

In [3]:
# !pip install -U "ray[rllib]"  

In [4]:
# !pip install -U "ray[serve]"  

In [120]:
# Imports
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!
from sklearn.model_selection import KFold

import numpy as np
import os
import pandas as pd

import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split
import torch.nn as nn



In [121]:
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from functools import partial

In [122]:
"""
(Hyper)parameters of our convolutional neural network found by tuning with Ray Tune (see further on)

"""
input_size = 1 * 110 * 40
num_classes = 3
learning_rate = 0.009378349953829784
batch_size = 10
num_epochs = 10
torch.manual_seed(42)

## Complete Convolutional Neural Network
A first Convolutional Neural Network that takes on one input channel, where the PAC-matrix will be feeded as an input, and returns a probability array of total length equal to the amount of label classes. Each probability equals the chance of the label to correspond to the label class.

The network first consists of 3 Convolutional layers each of kernel size 3x3. Stride is set to its default value 1 and padding is initialised to 1 as well.
After each convolutional layer: first, a rectified linear unit activation function is applied and second a max pooling layer of kernel size 2x2 is used.

After the last convolutional layer, the output is flattened and fed into a first fully connected neural network layer where a tanh activation function is applied its the output. After this a random dropout with probability of dropping 0.25 is performed.
Finally a batch normalization is performed.

This procedure gets repeated after two more fully connected layers until the tensor after the third fully connected layer has a size equal to the amount of label classes.

As a final step this the softmax funtion is applied to this tensor, which transforms each element to its respective probability.

This CNN was designed purely for tuning the hyperparameters and getting general insights. The accuracies are not reported.


In [123]:
"""
Class inherits from nn.Module, a pytorch neural network of choice, must inherit from base class nn.Module.
"""
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1,
        ) 
        
        self.conv3 = nn.Conv2d(
            in_channels=16,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        
        x = torch.randn(110, 40).view(-1, 1, 110, 40)
        self._to_linear = None
        
        self.dropout = nn.Dropout(0.25)
        self.convs(x)
        self.fc1 = nn.Linear(self._to_linear, 100) #flattening.
        self.fc2 = nn.Linear(100, 20)
        self.fc3 = nn.Linear(20, num_classes) 
        self.batchnorm1 = nn.BatchNorm1d(100)
        self.batchnorm2 = nn.BatchNorm1d(20)
        self.batchnorm3 = nn.BatchNorm1d(num_classes)     
    def convs(self, x):
        """
        Wrapper function for convolutional layers.
        
        self: self
        x: tensor of shape (num_channels, PAC_y_dim, PAC_x_dim)
            where PAC_y_dim and PAC_x_dim are the dimensions of the PAC-matrix
        
        returns: tensor of shape 'self._to_linear'
        """   
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        
        if self._to_linear == None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
            
        return x
    
    def forward(self, x):
        """
        Forward propagation function.
        
        self: self
        x: tensor of shape (num_channels, PAC_y_dim, PAC_x_dim)
            where PAC_y_dim and PAC_x_dim are the dimensions of the PAC-matrix
        
        returns: tensor of shape (1, 100) serving as a feature vector.
        """
        x = self.convs(x) #execute convolutional layers      
        x = x.view(-1, self._to_linear) #flatten
        x = F.tanh(self.fc1(x))#pass through first fc layer
        x = self.dropout(x)
        x = self.batchnorm1(x)
        x = F.tanh(self.fc2(x))#pass through first fc layer
        x = self.dropout(x)
        x = self.batchnorm2(x)
        x = self.fc3(x) #final fc layer
        x = self.dropout(x)
        x = self.batchnorm3(x)
        return F.softmax(x, dim=1)

## Convolutional Neural Network for Feature Extraction
In order to use the PAC-matrix as a feature for a Multi-Layer Perceptron later on, the previous Convolutional Neural Network is slightly adapted to not return the tensor with label probabilities, but a tensor of shape (1, 100).

The network first consists of 3 Convolutional layers each of exactly the same parameters as the previous network.

After the last convolutional layer, the output is flattened and fed into a first fully connected neural network layer where a tanh activation function is applied its the output. After this a random dropout with probability of dropping 0.25 is performed.
Finally a batch normalization is performed.

After this the resulting tensor gets returned as a feature vector, which is fed along with other features to the final MLP.

In [124]:
"""
Class inherits from nn.Module, a pytorch neural network of choice, must inherit from base class nn.Module.
"""
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1,
        ) 
        
        self.conv3 = nn.Conv2d(
            in_channels=16,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        
        x = torch.randn(110, 40).view(-1, 1, 110, 40)
        self._to_linear = None
        
        self.dropout = nn.Dropout(0.25)
        self.convs(x)
        self.fc1 = nn.Linear(self._to_linear, 100) #flattening.
        self.batchnorm1 = nn.BatchNorm1d(100)
        
    def convs(self, x):
        """
        Wrapper function for convolutional layers.
        
        self: self
        x: tensor of shape (num_channels, PAC_y_dim, PAC_x_dim)
            where PAC_y_dim and PAC_x_dim are the dimensions of the PAC-matrix
        
        returns: tensor of shape ?
        """
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        
        if self._to_linear == None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    def forward(self, x):
        """
        Forward propagation function.
        
        self: self
        x: tensor of shape (num_channels, PAC_y_dim, PAC_x_dim)
            where PAC_y_dim and PAC_x_dim are the dimensions of the PAC-matrix
        
        returns: tensor of shape (1, 100) serving as a feature vector.
        """
        x = self.convs(x) #execute convolutional layers
        x = x.view(-1, self._to_linear) #flatten
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.batchnorm1(x)
        return x

In [125]:
def reset_weights(m):
  """
  Try resetting model weights to avoid
  weight leakage.
    
  m: a PyTorch model
    
  return: None
  """
  for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            #print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [126]:
"""
Set device cuda for GPU if it's available otherwise run on the CPU
"""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load labels

In [129]:
"""
Load labels from csv file
"""
labels = np.genfromtxt('../Data/Labels/avg_smooth.csv', delimiter=',')

## Quantify raw label values
After reading the raw label values, they're quantified to 3 discrete levels: 0, 1 or 2.

0: 'No fear apparent in rodents brain activity'

1: 'A small amount of fear could be noticed, but too noisey to be sure.'

2: 'Rodent is fearful'

In [130]:
df_labels = pd.DataFrame(labels)
df_labels = pd.cut(df_labels[0],bins=[0, 0.013, 0.017, 0.03],labels=[0,1,2])
labels = df_labels.to_numpy()


print('\nAmount of non-fearful rodents:' + str(len([i for i in list(labels) if i==0])))
print('Amount of little fearful rodents:' + str(len([i for i in list(labels) if i==1])))
print('Amount of fearful rodents:' + str(len([i for i in list(labels) if i==2])))


Amount of non-fearful rodents:200
Amount of little fearful rodents:204
Amount of fearful rodents:196


## Load all PAC-matrices from .npy-files

In [131]:
"""
Load .npy files into one big array
"""
data = []
dir = '../Data/Feature arrays/'
for filename in os.listdir(dir):
    sample = np.load(dir+filename)
    data.append(sample[0]) #select only PAC
data = np.array(data[:-1])
data = np.expand_dims(data,1)
print('Feature shapes: ' + str(data.shape), '\nLabel vector shape: ' + str(labels.shape))

Feature shapes: (600, 1, 110, 40) 
Label vector shape: (600,)


In [132]:
"""
Transform data to pytorch tensors
"""
tensor_x = torch.Tensor(data)
tensor_y = torch.Tensor(labels)
tensor_y = tensor_y.type(torch.LongTensor)

In [133]:
"""
Create pytorch dataset and dataloader
"""
dataset = TensorDataset(tensor_x,tensor_y)

"""
Split into train and test sets
"""
test_size = int(0.3*len(dataset))
train_size = len(dataset) - test_size

train_data,test_data = random_split(dataset,[train_size,test_size])
print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Test Data : {len(test_data)}")

train_loader = DataLoader(train_data, batch_size, shuffle = False, num_workers = 2, pin_memory = True)
test_loader = DataLoader(test_data, batch_size, shuffle = False, num_workers = 2, pin_memory = True)

Length of Train Data : 420
Length of Test Data : 180


In [134]:
"""
Initialize network
""" 
model = CNN().to(device)

"""
Use CrossEntropyLoss and Adam optimizer
""" 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Define accuracy function

In [135]:
"""
 Check accuracy on training & test to see how good the model is
 
 loader: a pytorch dataloader
 model: a pytorch model
 
 returns: model accuracy as a numerical value
"""
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            #x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples

In [136]:
"""
 Check accuracy on training & test to see how good the model performs within each class
 
 loader: a pytorch dataloader
 model: a pytorch model
 
 returns: a list of model accuracies as a numerical value
"""
def check_accuracy_per_class(loader, model):
    num_correct = [0.0, 0.0, 0.0]
    num_samples = [0.0, 0.0, 0.0]
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            #x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            for (prediction, y_i) in zip(predictions, y):
                prediction = prediction.item()
                y_i = y_i.item()
                if(y_i == 0):
                    if(prediction == y_i): 
                        num_correct[0] += 1
                    num_samples[0] += 1
                elif(y_i == 1):
                    if(prediction == y_i): 
                        num_correct[1] += 1
                    num_samples[1] += 1
                elif(y_i == 2):
                    if(prediction == y_i): 
                        num_correct[2] += 1
                    num_samples[2] += 1
    model.train()
    accuracy = [0.0, 0.0, 0.0]
    if num_samples[0] != 0:
        accuracy[0] = num_correct[0]/num_samples[0]
    if num_samples[1] != 0:
        accuracy[1] = num_correct[1]/num_samples[1]
    if num_samples[2] != 0:
        accuracy[2] = num_correct[2]/num_samples[2]
    return accuracy

## Apply crossvalidation
Apply 5-fold crossvalidation with CrossEntropy loss and Adam optimizer.

In [137]:
"""
Define the 5-fold Cross Validator
""" 
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=False)

In [142]:
train_acc = []
test_acc = []

train_acc_per_class = []
test_acc_per_class = []

for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    print(f'Fold {fold}')
    model.apply(reset_weights)
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    trainloader = torch.utils.data.DataLoader(
                      dataset, batch_size=10, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset, batch_size=10, sampler=test_subsampler)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        for batch_idx, (input_, labels) in enumerate(tqdm(trainloader)):
            # Get data to cuda if possible
            input_ = input_.to(device=device)
            targets = labels.to(device=device)

            # forward
            scores = model(input_)
            loss = criterion(scores, targets)

            # backward
            optimizer.zero_grad()
            loss.backward()

            # gradient descent or adam step
            optimizer.step()
        train_acc.append(check_accuracy(trainloader, model).item()*100)
        test_acc.append(check_accuracy(testloader, model).item()*100)
        train_acc_per_class.append(check_accuracy_per_class(trainloader, model))
        test_acc_per_class.append(check_accuracy_per_class(testloader, model))
    print('Train Accuracy for fold %d: %d %%' % (fold, 100.0 * check_accuracy(trainloader, model)))
    print('Test Accuracy for fold %d: %d %%' % (fold, 100.0 * check_accuracy(testloader, model)))
    print('Train Accuracies per class [Label 0, Label 1, Label 2] for fold %d: ' % (fold))
    print(*check_accuracy_per_class(trainloader, model))
    print('Test Accuracies per class [Label 0, Label 1, Label 2] for fold %d: ' % (fold))
    print(*check_accuracy_per_class(testloader, model))
print('Averaged Train Accuracy over %d k-folds: %d %%' % (k_folds, np.array(train_acc).mean()))
print('Averaged Test Accuracy over %d k-folds: %d %%' % (k_folds, np.array(test_acc).mean()))
print('Averaged Train Accuracy per class', *np.matrix(train_acc_per_class).mean(0))
print('Averaged Test Accuracy per class', *np.matrix(test_acc_per_class).mean(0))

Fold 0


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 65.93it/s]


Train Accuracy for fold 0: 36 %
Test Accuracy for fold 0: 29 %
Train Accuracies per class [Label 0, Label 1, Label 2] for fold 0: 
0.04411764705882353 0.6011560693641619 0.3742690058479532
Test Accuracies per class [Label 0, Label 1, Label 2] for fold 0: 
0.03125 0.6774193548387096 0.48
Fold 1


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 55.05it/s]


Train Accuracy for fold 1: 39 %
Test Accuracy for fold 1: 29 %
Train Accuracies per class [Label 0, Label 1, Label 2] for fold 1: 
0.5 0.006896551724137931 0.6184971098265896
Test Accuracies per class [Label 0, Label 1, Label 2] for fold 1: 
0.631578947368421 0.0 0.4782608695652174
Fold 2


...

Train Accuracy for fold 2: 44 %
Test Accuracy for fold 2: 15 %
Train Accuracies per class [Label 0, Label 1, Label 2] for fold 2: 
0.30714285714285716 0.013245033112582781 0.8888888888888888
Test Accuracies per class [Label 0, Label 1, Label 2] for fold 2: 
0.21666666666666667 0.0 0.7142857142857143
Fold 3


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 58.82it/s]


Train Accuracy for fold 3: 39 %
Test Accuracy for fold 3: 10 %
Train Accuracies per class [Label 0, Label 1, Label 2] for fold 3: 
0.9206349206349206 0.08888888888888889 0.0
Test Accuracies per class [Label 0, Label 1, Label 2] for fold 3: 
0.8181818181818182 0.125 0.0
Fold 4


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 56.07it/s]


Train Accuracy for fold 4: 39 %
Test Accuracy for fold 4: 32 %
Train Accuracies per class [Label 0, Label 1, Label 2] for fold 4: 
0.6473988439306358 0.2694610778443114 0.2357142857142857
Test Accuracies per class [Label 0, Label 1, Label 2] for fold 4: 
0.5185185185185185 0.35135135135135137 0.21428571428571427
Averaged Train Accuracy over 5 k-folds: 37 %
Averaged Test Accuracy over 5 k-folds: 25 %
Averaged Train Accuracy per class [[0.38062977 0.27790589 0.41296269]]
Averaged Test Accuracy per class [[0.350379   0.26318976 0.34706142]]


## Save extracted feature vector to .npy file to feed it into MLP

In [42]:
features = model.forward(tensor_x.to(device=device))
features = features.cpu().detach().numpy()
features = np.mean(features, axis=0)
np.save('../Data/PAC_afterCNN.npy', features)

(pid=None) [2021-12-22 15:37:05,707 C 32412 23428] redis_client.cc:87:  Check failed: under_retry_limit Expected 1 Redis shard addresses, found 2
(pid=None) *** StackTrace Information ***
(pid=None)     configthreadlocale
(pid=None)     BaseThreadInitThunk
(pid=None)     RtlUserThreadStart
(pid=None) 
(pid=None) [2021-12-22 15:38:25,202 C 21372 30712] redis_client.cc:87:  Check failed: under_retry_limit Expected 1 Redis shard addresses, found 3
(pid=None) *** StackTrace Information ***
(pid=None)     configthreadlocale
(pid=None)     BaseThreadInitThunk
(pid=None)     RtlUserThreadStart
(pid=None) 
100%|██████████| 4/4 [00:05<00:00,  1.31s/it]


 pid=14084) Finished


100%|██████████| 4/4 [00:04<00:00,  1.23s/it]


 pid=23468) Finished


100%|██████████| 18/18 [00:04<00:00,  3.68it/s]


 pid=31648) Finished


## Hyperparameter Tuning with Ray Tune
As noted before some parameters where acquired in their optimal form by performing hyperparameter tuning.  
The library used for this was Ray Tune.  

Following hyperparameters where tuned:
- number of epochs
- learning rate
- batch size

with following optimal values:  
- number of epochs = 10
- learning_rate = 0.009378349953829784
- batch_size = 10


In [20]:
config = {
    "num_epochs": tune.choice([2, 5, 10]),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([5, 10, 50, 100])
}
max_num_epochs = 11

In [21]:
"""
Ray Tune requires one method to contain all necessary network definitions and methods.

config: dictionary with configuration hyperparameters
checkpoint_dir: not used, but could be used to load earlier configuration

returns: None
"""
def train(config, checkpoint_dir=None):
    model = CNN().to(device)
    model.apply(reset_weights)
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])
    
    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
    batch_size = config["batch_size"]
    train_loader = DataLoader(train_data, batch_size, shuffle = False, num_workers = 2, pin_memory = True)
    test_loader = DataLoader(test_data, batch_size, shuffle = False, num_workers = 2, pin_memory = True)
    for epoch in range(config["num_epochs"]):
        running_loss = 0
        epoch_steps = 0
        for batch_idx, (data, labels) in enumerate(tqdm(train_loader)):
            current_loss = 0
            # Get data to cuda if possible
            data = data.to(device=device)
            targets = labels.to(device=device)

            # forward
            scores = model(data)
            loss = criterion(scores, targets)

            # backward
            optimizer.zero_grad()
            loss.backward()

            # gradient descent or adam step
            optimizer.step()

            #print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if batch_idx%500 == 499:
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0
                
                
        test_loss = 0
        test_steps = 0
        total = 0
        correct = 0
        for batch_idx, (data, labels) in enumerate(tqdm(test_loader)):
            with torch.no_grad():
                data = data.to(device=device)
                targets = labels.to(device=device)
                scores = model(data)
                _, predictions = scores.max(1)
                total += predictions.size(0)
                correct += (predictions == targets).sum().item()
                loss = criterion(scores, targets)
                test_loss += loss.cpu().numpy()
                test_steps+=1
        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)
        tune.report(loss=(test_loss/test_steps), accuracy=correct/total)
    print("Finished")

            
    

In [22]:
"""
num_samples: amount of hyperparameter value combinations to check
gpus_per_trial: amount of gpus available

return: None
"""
def main(num_samples = 10, gpus_per_trial=1):
    config = {
    "num_epochs": tune.choice([4, 5, 10]),
    "learning_rate": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([10, 30, 50, 100])
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t = max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(metric_columns=["loss", "accuray", "train_iteration"])
    result = tune.run(
            partial(train),
            resources_per_trial={"cpu": 1, "gpu": gpus_per_trial},
            config=config,
            num_samples = num_samples,
            scheduler=scheduler,
            progress_reporter=reporter)
    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))
    #best_trained_model = model(best_trial)

In [24]:
main(num_samples=10, gpus_per_trial=1)

2021-12-22 15:23:21,471	WARNING experiment.py:255 -- No name detected on trainable. Using DEFAULT.
2021-12-22 15:23:21,472	INFO registry.py:69 -- Detected unknown callable for trainable. Converting to class.
2021-12-22 15:23:31,356	WARNING worker.py:1245 -- Warning: The actor ImplicitFunc is very large (10 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
2021-12-22 15:23:31,573	WARNING util.py:165 -- The `start_trial` operation took 1.050 s, which may be a performance bottleneck.


== Status ==
Current time: 2021-12-22 15:23:31 (running for 00:00:09.63)
Memory usage on this node: 16.8/47.6 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 1.0/12 CPUs, 1.0/1 GPUs, 0.0/19.14 GiB heap, 0.0/9.57 GiB objects
Result logdir: C:\Users\Tomas\ray_results\DEFAULT_2021-12-22_15-23-21
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+---------------------+----------+-----------------+--------------+-----------------+--------------+
| Trial name          | status   | loc             |   batch_size |   learning_rate |   num_epochs |
|---------------------+----------+-----------------+--------------+-----------------+--------------|
| DEFAULT_b77bd_00000 | RUNNING  | 127.0.0.1:30604 |           10 |     0.00937835  |           10 |
| DEFAULT_b77bd_00001 | PENDING  |                 |           30 |     0.000402305 |            5 |
| DEFAULT_b77bd_00002 | PENDING  |                 |      

2021-12-22 15:23:35,410	WARNING util.py:165 -- The `start_trial` operation took 0.786 s, which may be a performance bottleneck.
2021-12-22 15:23:36,255	WARNING util.py:165 -- The `start_trial` operation took 0.716 s, which may be a performance bottleneck.
2021-12-22 15:23:37,002	WARNING util.py:165 -- The `start_trial` operation took 0.634 s, which may be a performance bottleneck.


== Status ==
Current time: 2021-12-22 15:23:37 (running for 00:00:15.15)
Memory usage on this node: 17.4/47.6 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 1.0/12 CPUs, 1.0/1 GPUs, 0.0/19.14 GiB heap, 0.0/9.57 GiB objects
Result logdir: C:\Users\Tomas\ray_results\DEFAULT_2021-12-22_15-23-21
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+---------------------+----------+-----------------+--------------+-----------------+--------------+
| Trial name          | status   | loc             |   batch_size |   learning_rate |   num_epochs |
|---------------------+----------+-----------------+--------------+-----------------+--------------|
| DEFAULT_b77bd_00000 | RUNNING  | 127.0.0.1:30604 |           10 |     0.00937835  |           10 |
| DEFAULT_b77bd_00001 | PENDING  |                 |           30 |     0.000402305 |            5 |
| DEFAULT_b77bd_00002 | PENDING  |                 |      

2021-12-22 15:23:38,198	WARNING util.py:165 -- The `start_trial` operation took 1.084 s, which may be a performance bottleneck.
2021-12-22 15:23:39,067	WARNING util.py:165 -- The `start_trial` operation took 0.751 s, which may be a performance bottleneck.
2021-12-22 15:23:39,836	WARNING util.py:165 -- The `start_trial` operation took 0.650 s, which may be a performance bottleneck.
  0%|          | 0/42 [00:00<?, ?it/s]
2021-12-22 15:23:40,765	WARNING util.py:165 -- The `start_trial` operation took 0.820 s, which may be a performance bottleneck.
2021-12-22 15:23:42,378	WARNING util.py:165 -- The `start_trial` operation took 0.594 s, which may be a performance bottleneck.


...

== Status ==
Current time: 2021-12-22 15:29:56 (running for 00:06:34.62)
Memory usage on this node: 17.8/47.6 GiB
Using AsyncHyperBand: num_stopped=8
Bracket: Iter 8.000: -0.8603654172685411 | Iter 4.000: -0.8872017562389374 | Iter 2.000: -1.066592276096344 | Iter 1.000: -1.1103543043136597
Resources requested: 1.0/12 CPUs, 1.0/1 GPUs, 0.0/19.14 GiB heap, 0.0/9.57 GiB objects
Result logdir: C:\Users\Tomas\ray_results\DEFAULT_2021-12-22_15-23-21
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+---------------------+------------+-----------------+--------------+-----------------+--------------+----------+
| Trial name          | status     | loc             |   batch_size |   learning_rate |   num_epochs |     loss |
|---------------------+------------+-----------------+--------------+-----------------+--------------+----------|
| DEFAULT_b77bd_00009 | RUNNING    | 127.0.0.1:24060 |           50 |     0.000146973 |            5 |          |
| DEFAULT_b77bd_00000 | TERMINATED | 127.0.0.

 pid=24060) C:\Users\Tomas\anaconda3\lib\site-packages\torch\nn\functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
 pid=24060)   warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
  0%|          | 0/4 [00:00<?, ?it/s]


== Status ==
Current time: 2021-12-22 15:30:06 (running for 00:06:44.74)
Memory usage on this node: 18.7/47.6 GiB
Using AsyncHyperBand: num_stopped=8
Bracket: Iter 8.000: -0.8603654172685411 | Iter 4.000: -0.8872017562389374 | Iter 2.000: -1.066592276096344 | Iter 1.000: -1.1103543043136597
Resources requested: 1.0/12 CPUs, 1.0/1 GPUs, 0.0/19.14 GiB heap, 0.0/9.57 GiB objects
Result logdir: C:\Users\Tomas\ray_results\DEFAULT_2021-12-22_15-23-21
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+---------------------+------------+-----------------+--------------+-----------------+--------------+----------+
| Trial name          | status     | loc             |   batch_size |   learning_rate |   num_epochs |     loss |
|---------------------+------------+-----------------+--------------+-----------------+--------------+----------|
| DEFAULT_b77bd_00009 | RUNNING    | 127.0.0.1:24060 |           50 |     0.000146973 |            5 |          |
| DEFAULT_b77bd_00000 | TERMINATED | 127.0.0.

100%|██████████| 4/4 [00:04<00:00,  1.20s/it]
2021-12-22 15:30:08,173	INFO tune.py:626 -- Total run time: 406.71 seconds (406.08 seconds for the tuning loop).


Result for DEFAULT_b77bd_00009:
  accuracy: 0.36666666666666664
  date: 2021-12-22_15-30-08
  done: true
  experiment_id: 91639c695ecd4f229c156f143d2cc71d
  hostname: WSTOMAS21
  iterations_since_restore: 1
  loss: 1.1123794317245483
  node_ip: 127.0.0.1
  pid: 24060
  should_checkpoint: true
  time_since_restore: 17.335869789123535
  time_this_iter_s: 17.335869789123535
  time_total_s: 17.335869789123535
  timestamp: 1640183408
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: b77bd_00009
  
== Status ==
Current time: 2021-12-22 15:30:08 (running for 00:06:46.09)
Memory usage on this node: 18.6/47.6 GiB
Using AsyncHyperBand: num_stopped=9
Bracket: Iter 8.000: -0.8603654172685411 | Iter 4.000: -0.8872017562389374 | Iter 2.000: -1.066592276096344 | Iter 1.000: -1.111366868019104
Resources requested: 0/12 CPUs, 0/1 GPUs, 0.0/19.14 GiB heap, 0.0/9.57 GiB objects
Result logdir: C:\Users\Tomas\ray_results\DEFAULT_2021-12-22_15-23-21
Number of trials: 10/10 (10 TERMINATED)
+--